<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/pytorch/101/pytorch_cnn_experiment_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://blog.algorithmia.com/convolutional-neural-nets-in-pytorch

import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNetCustomTest(nn.Module):
    def __init__(self, num_classes=10):
      super(ConvNetCustomTest, self).__init__()

      #Input channels = 3, output channels = 18
      self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
      self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

      
      #4608 input features, 64 output features (see sizing flow below)
      self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
      #64 input features, 10 output features for our 10 defined classes
      self.fc2 = torch.nn.Linear(64, 10)

    def forward(self,x) :
      x = F.relu(self.conv1(x))
      #Size changes from (3, 32, 32) to (18, 32, 32)

      x = self.pool(x)
      #Size changes from (18, 32, 32) to (18, 16, 16)

      #Reshape data to input to the input layer of the neural net
      #Size changes from (18, 16, 16) to (1, 4608)
      #Recall that the -1 infers this dimension from the other given dimension
      x = x.view(-1, 18 * 16 *16)

      x = F.relu(self.fc1(x))

      x = self.fc2(x)
      return(x)
      

    

In [2]:

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)

test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)





0it [00:00, ?it/s]

170500096it [00:07, 24296055.90it/s]                               


Extracting ./cifardata/cifar-10-python.tar.gz to ./cifardata
Files already downloaded and verified


In [3]:
print(train_set[0][0].shape)

print(len(train_set))

torch.Size([3, 32, 32])
50000


In [0]:
from torch.utils.data.dataset import random_split

train_dataset, val_dataset, test_dataset, _ = random_split(train_set, [800, 150, 50, 49000])


In [0]:

test_loader = torch.utils.data.DataLoader(test_set, batch_size=50, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=50, num_workers=2)



In [0]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [7]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,num_workers=2)

    #Get training data
    # train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):

            # print(data[0])
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            # inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics

            running_loss += loss_size.item()
            total_train_loss += loss_size.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
                        
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.item()
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

CNN = ConvNetCustomTest()
trainNet(CNN, batch_size=32, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 5
learning_rate= 0.001
Epoch 1, 12% 	 train_loss: 3.39 took: 0.53s
Epoch 1, 24% 	 train_loss: 3.46 took: 0.06s
Epoch 1, 36% 	 train_loss: 3.34 took: 0.07s
Epoch 1, 48% 	 train_loss: 3.31 took: 0.07s
Epoch 1, 60% 	 train_loss: 3.24 took: 0.08s
Epoch 1, 72% 	 train_loss: 3.11 took: 0.07s
Epoch 1, 84% 	 train_loss: 3.29 took: 0.07s
Epoch 1, 96% 	 train_loss: 3.20 took: 0.05s
Validation loss = 2.08
Epoch 2, 12% 	 train_loss: 2.88 took: 0.14s
Epoch 2, 24% 	 train_loss: 3.09 took: 0.06s
Epoch 2, 36% 	 train_loss: 2.91 took: 0.07s
Epoch 2, 48% 	 train_loss: 3.04 took: 0.07s
Epoch 2, 60% 	 train_loss: 2.90 took: 0.07s
Epoch 2, 72% 	 train_loss: 2.81 took: 0.07s
Epoch 2, 84% 	 train_loss: 3.08 took: 0.07s
Epoch 2, 96% 	 train_loss: 3.05 took: 0.06s
Validation loss = 1.98
Epoch 3, 12% 	 train_loss: 2.62 took: 0.14s
Epoch 3, 24% 	 train_loss: 3.08 took: 0.07s
Epoch 3, 36% 	 train_loss: 2.58 took: 0.06s
Epoch 3, 48% 	 train_loss: 2.73 took: 0.06s


In [9]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 34.47 %
